In [1]:

import zipfile
import os

# Step 2: Define paths
zip_path = r'C:\Users\priya\Desktop/Iris_Recognition/Gpt/IITD_database-20250625T182513Z-1-001.zip'
extract_path = r'C:\Users\priya\Desktop\Gpt\Extracted'  # Destination after unzipping

# Step 3: Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")

Extraction complete.


In [2]:
! pip install tqdm


In [3]:
import os
import shutil
import random

def get_all_images_recursive(class_path):
    images = []
    for root, _, files in os.walk(class_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                images.append(os.path.join(root, file))
    return images

def split_dataset_clientwise(input_dir, output_base_dir, num_clients=3, classes_per_client=74, train_ratio=0.625, val_ratio=0.25, test_ratio=0.125):
    random.seed(42)

    # Get all valid class folders
    class_folders = sorted([
        folder for folder in os.listdir(input_dir)
        if os.path.isdir(os.path.join(input_dir, folder))
    ])

    total_required_classes = classes_per_client * num_clients
    if len(class_folders) < total_required_classes:
        raise ValueError(f"Not enough classes in the dataset. Required: {total_required_classes}, Found: {len(class_folders)}")

    selected_classes = class_folders[:total_required_classes]

    # Shuffle for randomness
    random.shuffle(selected_classes)

    client_class_map = {}
    for i in range(num_clients):
        start = i * classes_per_client
        end = start + classes_per_client
        client_class_map[i + 1] = selected_classes[start:end]

    for client_id, class_list in client_class_map.items():
        print(f"Processing Client {client_id} with {len(class_list)} classes...")
        output_dir = os.path.join(output_base_dir, f"client{client_id}")
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        os.makedirs(output_dir, exist_ok=True)

        for class_name in class_list:
            class_path = os.path.join(input_dir, class_name)
            images = get_all_images_recursive(class_path)
            if not images:
                print(f"Skipping empty class folder: {class_path}")
                continue
            random.shuffle(images)

            total = len(images)
            train_end = int(train_ratio * total)
            val_end = train_end + int(val_ratio * total)

            train_imgs = images[:train_end]
            val_imgs = images[train_end:val_end]
            test_imgs = images[val_end:]

            for split_name, split_imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
                split_dir = os.path.join(output_dir, split_name, class_name)
                os.makedirs(split_dir, exist_ok=True)
                for img_path in split_imgs:
                    img_name = os.path.basename(img_path)
                    dst = os.path.join(split_dir, img_name)
                    shutil.copy2(img_path, dst)

if __name__ == "__main__":
    input_dataset_folder = r'C:\Users\priya\Desktop\Gpt\Extracted\IITD_database'
    output_base_folder = r'C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset2\Clients'

    split_dataset_clientwise(input_dataset_folder, output_base_folder)


Processing Client 1 with 74 classes...
Processing Client 2 with 74 classes...
Processing Client 3 with 74 classes...


In [1]:
import os
import shutil
from glob import glob

def merge_test_datasets(input_dirs, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for input_dir in input_dirs:
        for class_name in os.listdir(input_dir):
            class_input_path = os.path.join(input_dir, class_name)
            class_output_path = os.path.join(output_dir, class_name)

            if not os.path.isdir(class_input_path):
                continue

            os.makedirs(class_output_path, exist_ok=True)

            for file_path in glob(os.path.join(class_input_path, "*.*")):
                # Create a unique filename to avoid name collisions
                file_name = os.path.basename(file_path)
                new_file_name = f"{os.path.basename(input_dir)}_{file_name}"
                dest_path = os.path.join(class_output_path, new_file_name)
                shutil.copy(file_path, dest_path)

    print(f"✅ Merged datasets saved to: {output_dir}")


In [ ]:
import os
merge_test_datasets(
    input_dirs=[
        r"",
        r"",
        r""
    ],
    output_dir=r"C:\Users\priya\Desktop\Iris_Recognition\test_combined"
)
